In [2]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy as np
import tflearn
import tensorflow as tf
import random

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [8]:
import json
with open('intents.json') as json_data:
    d = json.load(json_data)
    print(d)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'], 'responses': ['Hello, thanks for visiting', 'Good to see you again', 'Hi there, how can I help?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye'], 'responses': ['See you later, thanks for asking', 'Have a nice day', 'Bye! Come back again.']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful"], 'responses': ['Happy to help!', 'Any time!', 'My pleasure']}, {'tag': 'Games', 'patterns': ['What are all games can play', 'What games are there?', 'I just want to know game events?', 'Can you provide me link to register games?'], 'responses': ['See the list of games and the links to register ', 'list of games 1. basket ball 2. volley ball 3.badminton 4.snookers']}, {'tag': 'food', 'patterns': ['What special food on foundation day?', 'What kinds of foods are there on foundation day?', 'What do you sell?'], 'responses': ['keep you

In [9]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in d['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

26 documents
7 classes ['Games', 'food', 'goodbye', 'greeting', 'location', 'thanks', 'yes']
53 unique stemmed words ['!', "'s", 'al', 'anyon', 'ar', 'bye', 'can', 'celebr', 'day', 'do', 'ev', 'food', 'found', 'gam', 'going', 'good', 'goodby', 'hello', 'help', 'hi', 'hmm', 'how', 'i', 'is', 'just', 'kind', 'know', 'lat', 'link', 'loc', 'me', 'of', 'ohh', 'on', 'play', 'provid', 'reg', 'see', 'sel', 'spec', 'thank', 'that', 'the', 'ther', 'to', 'want', 'we', 'what', 'wher', 'wow', 'ye', 'yeah', 'you']


In [10]:
training = []
output = []

output_empty = [0] * len(classes)

for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [11]:
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 3999  | total loss: 0.04452 | time: 0.012s
| Adam | epoch: 1000 | loss: 0.04452 - acc: 1.0000 -- iter: 24/26
Training Step: 4000  | total loss: 0.04728 | time: 0.016s
| Adam | epoch: 1000 | loss: 0.04728 - acc: 1.0000 -- iter: 26/26
--
INFO:tensorflow:C:\Users\Hemanth\Desktop\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [12]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [14]:
p = bow("what are the food available", words)
print (p)
print (classes)

[0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0]
['Games', 'food', 'goodbye', 'greeting', 'location', 'thanks', 'yes']


In [15]:
print(model.predict([p]))

[[  1.39444877e-04   5.20549953e-01   7.60175809e-02   3.33626382e-02
    3.68634701e-01   1.02766883e-03   2.68067437e-04]]


In [16]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [17]:
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from C:\Users\Hemanth\Desktop\model.tflearn


In [18]:
context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in d['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return print(random.choice(i['responses']))

            results.pop(0)

In [21]:
classify('what is the food today?')

[('location', 0.66952282), ('food', 0.31880146)]

In [22]:
response('hello')

Hi there, how can I help?


In [23]:
response('hi')

Hello, thanks for visiting


In [64]:
response('what are all the games available on foundation day')

Are you ok with chennai trabe center?
